In [4]:
import sys
sys.path.append('/home/steidani/projects/unibe_forestfire/code/fire_indices')
import firedanger

import pandas as pd
import numpy as np
import zipfile

ModuleNotFoundError: No module named 'indices'

In [2]:
# load data from Angelis et al. 2015
datapath = '/home/steidani/windows/data/forest_fire/angelis_2015/'
zf = zipfile.ZipFile(datapath + 'angelis_2015.zip') 
df = pd.read_csv(zf.open('data/sa_1991_2012.csv'))

In [3]:
df

,Date,fire_days,T,H,U,P,CloudCover,SnowCover,WeekRain,DaysSinceRain,...,ISI,BUI,FWI,FFWI,KBDIsi,FFDI,Munger,Orieuxdanger,Nesterov,FMI
0,19910501,0,10.9,79.4,0.9,16.7,0.96,0,21.7,0,...,2.804555e-03,17.893613,2.371713e-03,2.057458,27.018808,0.108653,0.0,0,0.000000,27.125
1,19910502,0,10.1,85.4,1.4,8.8,1.00,0,27.9,0,...,2.657373e-03,11.541490,1.734838e-03,2.307935,19.073921,0.079698,0.0,0,0.000000,28.825
2,19910503,0,12.6,52.4,2.1,0.0,0.67,0,27.8,1,...,7.404163e-01,14.509640,5.515712e-01,7.691725,20.950458,0.463803,0.5,0,209.772503,19.950
3,19910504,0,8.2,53.7,2.6,20.2,0.92,0,48.0,0,...,2.355588e-01,8.249345,1.284055e-01,9.251411,0.779257,0.377870,0.0,0,0.000000,21.375
4,19910505,0,7.2,72.0,1.7,0.0,0.79,0,45.7,1,...,8.891783e-01,9.693415,5.274771e-01,4.491942,0.599977,0.236893,0.5,0,80.089967,26.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4703,20121126,0,7.9,94.1,0.8,40.8,1.00,0,42.3,0,...,1.278501e-06,2.408329,4.186612e-07,0.794523,0.289261,0.010714,0.0,0,0.000000,31.550
4704,20121127,0,7.2,96.2,0.9,74.6,1.00,0,116.9,0,...,9.662362e-09,0.509294,2.282898e-09,0.700693,0.263868,0.002818,0.0,0,0.000000,32.250
4705,20121128,0,6.6,95.4,0.7,48.2,1.00,0,165.1,0,...,1.433281e-08,0.000000,2.866562e-09,0.626805,0.109220,0.001871,0.0,0,0.000000,32.200
4706,20121129,0,7.8,59.3,1.6,4.5,0.88,0,169.4,0,...,1.240317e-03,0.000000,2.480633e-04,5.483013,0.494553,0.007344,0.0,0,0.000000,22.875


In [4]:
# first test, calculate FWI components
for index, row in df.iloc[1:10].iterrows():
    ffmc = firedanger.ffmc(row['T'],row['P'],row['U'],row['H'],df.loc[index-1]['FFMC'])
    print(ffmc, "     vs     ", row['FFMC'])

19.993972015769625      vs      28.124983801
57.510663752595526      vs      64.0041470655
39.51477068668489      vs      48.9754818157
61.49366325442538      vs      67.3922612489
37.689784616810975      vs      52.6305965266
68.21879136626956      vs      71.735990304
78.79514639451124      vs      83.0328036207
35.396926546356966      vs      44.8466856805
18.96292042447449      vs      16.9032266206


what is happening here?  
the meteorological input T, U and H are daily mean. P is the 24h accumulated value from 6am to 5am (next day).
But we should have the observations at 12am (noon)

In [5]:
# read hourly meteoswiss data for Locarno - Monti (OTL)
zf = zipfile.ZipFile(datapath + 'order95434.zip') 
df_mch = pd.read_csv(zf.open('order_95434_data.txt'), sep=";")
df_mch.columns = ["stn", "time", "T", "P", "H", "U"]

In [6]:
# convert column "time" to  type datetime
df_mch['time'] = pd.to_datetime(df_mch['time'], format='%Y%m%d%H', errors='coerce')

In [7]:
# daily mean
df_avg = df_mch.resample('D', on='time').mean()
# for precip, we want the 24h accumulated at noon
df_avg['P'] = df_mch[df_mch.time.dt.hour.eq(12)]['P'].values
df_avg

,T,P,H,U
time,,,,
1991-05-01,10.920833,4.5,79.370833,0.850000
1991-05-02,10.066667,12.8,85.383333,1.437500
1991-05-03,12.608333,8.2,52.416667,2.050000
1991-05-04,8.212500,0.0,53.712500,2.629167
1991-05-05,7.204167,20.2,71.970833,1.725000
...,...,...,...,...
1991-11-26,6.470833,0.0,70.433333,0.641667
1991-11-27,6.591667,0.0,72.075000,0.662500
1991-11-28,6.737500,0.0,71.691667,0.541667


Looks good for T, H and U: same values as from WSL

In [8]:
# Now for Fire Index, we need all variables messured at noon

In [13]:
df_noon = df_mch[df_mch.time.dt.hour.eq(12)]
df_noon = df_noon.reset_index()
#df_noon['P'] = df_mch[df_mch.time.dt.hour.eq(18)]['P'].values
df_noon

,index,stn,time,T,P,H,U
0,12,OTL,1991-05-01 12:00:00,10.9,4.5,83.6,2.4
1,36,OTL,1991-05-02 12:00:00,11.4,12.8,75.3,1.6
2,60,OTL,1991-05-03 12:00:00,16.0,8.2,45.7,2.0
3,84,OTL,1991-05-04 12:00:00,9.6,0.0,42.0,2.8
4,108,OTL,1991-05-05 12:00:00,10.5,20.2,59.2,4.3
...,...,...,...,...,...,...,...
209,5028,OTL,1991-11-26 12:00:00,9.1,0.0,58.6,1.1
210,5052,OTL,1991-11-27 12:00:00,9.3,0.0,62.3,1.5
211,5076,OTL,1991-11-28 12:00:00,9.3,0.0,64.4,1.3
212,5100,OTL,1991-11-29 12:00:00,10.1,0.0,55.9,1.0


In [10]:
# I do not understand which precip value is used.
# To get somwhat similar results, we need to take the precip value at index + 1

In [17]:
# first test, calculate FWI components
ffmc0 = 85
dc0 = 15
dmc0 = 6

for index, row in df_noon.iloc[0:15].iterrows():
    print(index, ": ", row['time'])
    #print(row['T'],df.iloc[index]['P'],row['U'],row['H'],ffmc0)
    ffmc = firedanger.ffmc(t=row['T'],p=row['P'],w=row['U'],h=row['H'],ffmc0=ffmc0)
    ffmc0 = ffmc
    dc = firedanger.dc(t=row['T'],p=row['P'],mth=row['time'].month,lat=46.01,dc0=dc0)
    dc0 = dc
    dmc = firedanger.dmc(t=row['T'],p=row['P'],h=row['H'],mth=row['time'].month,lat=46.01,dmc0=dmc0)
    dmc0 = dmc
    isi = firedanger.isi(w=row['U'],ffmc=ffmc)
    bui = firedanger.bui(dmc, dc)
    fwi = firedanger.fwi(isi,bui)
    print("=================ffmc   ", ffmc)
    print("=================dc   ", dc)
    print("=================dmc   ", dmc)
    print("=================isi   ", isi)
    print("=================bui   ", bui)
    print("=================fwi   ", fwi)

0 :  1991-05-01 12:00:00
=================ffmc    47.927435513623145
=================dc    14.359689463861724
=================dmc    3.66289870271719
=================isi    0.19430255274839675
=================bui    4.4732092152722185
=================fwi    0.0797304299505686
1 :  1991-05-02 12:00:00
=================ffmc    30.11696441498087
=================dc    4.4559999999999995
=================dmc    2.0305214288453484
=================isi    0.004757470165052979
=================bui    1.9705455845547273
=================fwi    0.001467044130931227
2 :  1991-05-03 12:00:00
=================ffmc    47.96646920153058
=================dc    5.284
=================dmc    2.844597101072322
=================isi    0.1816583331905473
=================bui    2.708525950523819
=================fwi    0.06179473030651416
3 :  1991-05-04 12:00:00
=================ffmc    70.38907012447609
=================dc    9.416
=================dmc    4.478425897072322
=================isi    1

In [31]:
# Calculate more infices
nes0 = 0
mun0 = 0
for index, row in df_noon.iloc[0:15].iterrows():
    print(index, ": ", row['time'])
    print(row['T'],row['P'],row['U'],row['H'])
    tdew = firedanger.tdewpoint_from_relhum(t=row['T'], h=row['H'])
    ang = firedanger.angstroem(t=row['T'], h=row['H'])
    nes = firedanger.nesterov(t=row['T'], h=row['H'], p=row['P'], prev=nes0)
    nes0 = nes
    mun = firedanger.munger(p=row['P'], prev=mun0)
    mun0 = mun
    sharp = firedanger.fuel_moisture_index(t=row['T'], h=row['H']) 
    print("=================tdew   ", tdew)
    print("=================ang   ", ang)
    print("=================nes   ", nes)
    print("=================mun   ", mun)
    print("=================FMI   ", sharp)

0 :  1991-05-01 12:00:00
10.9 4.5 2.4 83.6
=================tdew    8.233856848021825
=================ang    5.79
=================nes    0.0
=================mun    0.0
=================FMI    28.174999999999997
1 :  1991-05-02 12:00:00
11.4 12.8 1.6 75.3
=================tdew    7.186820825989405
=================ang    5.324999999999999
=================nes    0.0
=================mun    0.0
=================FMI    25.975
2 :  1991-05-03 12:00:00
16.0 8.2 2.0 45.7
=================tdew    4.296940312659832
=================ang    3.3850000000000002
=================nes    0.0
=================mun    0.0
=================FMI    17.425
3 :  1991-05-04 12:00:00
9.6 0.0 2.8 42.0
=================tdew    -2.6886277932049634
=================ang    3.84
=================nes    117.97082681476763
=================mun    0.5
=================FMI    18.1
4 :  1991-05-05 12:00:00
10.5 20.2 4.3 59.2
=================tdew    2.8751310165646746
=================ang    4.609999999999999
========

In [61]:
pweek = df_noon['P'].rolling(min_periods=1, window=7).sum()
pannual = df_noon['P'].sum()/len(df_noon['P'])*365.25 # weil nur von mai - nov


2074.2445093457945

In [70]:
# Calculate more infices
psum0 = 0
pday0 = 0
kbdi0 = 0
for index, row in df_noon.iloc[0:15].iterrows():
    print(index, ": ", row['time'])
    print(psum0,kndi0)
    psum = firedanger.rain_sum(p=row['P'], prev=psum0)
    psum0 = psum
    pday = firedanger.days_since_rain(p=row['P'], prev=pday0, threshold=1)
    pday0 = pday
    kbdi = firedanger.kbdi(tmax=row['T'], p=row['P'], kbdi0=kbdi0, psum0=psum, pweek=pweek[index], pWeekThreshold=30, pAnnualAvg=pannual)
    kbdi0 = kbdi
    print("=================psum   ", psum)
    print("=================pday   ", pday)
    print("=================kbdi   ", kbdi)

0 :  1991-05-01 12:00:00
0 0.277600744425854
=================psum    4.5
=================pday    0
=================kbdi    0.5637933516181622
1 :  1991-05-02 12:00:00
4.5 0.277600744425854
=================psum    17.3
=================pday    0
=================kbdi    0.6471593685042923
2 :  1991-05-03 12:00:00
17.3 0.277600744425854
=================psum    25.5
=================pday    0
=================kbdi    1.6122583409856268
3 :  1991-05-04 12:00:00
25.5 0.277600744425854
=================psum    0
=================pday    1
=================kbdi    1.972739347407158
4 :  1991-05-05 12:00:00
0 0.277600744425854
=================psum    20.2
=================pday    0
=================kbdi    0.49967738042726734
5 :  1991-05-06 12:00:00
20.2 0.277600744425854
=================psum    0
=================pday    1
=================kbdi    0.6815405041686924
6 :  1991-05-07 12:00:00
0 0.277600744425854
=================psum    7.0
=================pday    0
=================kb

In [ ]:
# Store input data that can be used by WSL firecalculater
df_noon = df_noon.rename(columns={"time": "DateYYYYMMDD"})
df_noon.DateYYYYMMDD = df_noon.DateYYYYMMDD.dt.strftime('%Y%m%d')

df_noon.to_csv('/home/steidani/windows/data/forest_fire/test.csv', index=False)

## NEW TEST with data from Canton Bern
Here also strange: sometimes exact same value, sometimes different....

In [2]:
datapath = "/home/steidani/windows/data/forest_fire/obs/"
df_bern = pd.read_csv(datapath + 'NGA_20210000_DD_BE_CFWIS_Warnung_2011-2020.csv', sep=",")

In [3]:
# convert column "time" to  type datetime
df_bern['Datum'] = pd.to_datetime(df_bern['Datum'], format='%Y-%m-%d', errors='coerce')
df_bern = df_bern.rename(columns={"T[�C]": "T", "rH[%]": "H", "P[mm/24h]": "P", "Vw[m/s]": "U"})

# keep only with year 2019
df_bern = df_bern[df_bern.Datum.dt.year.eq(2018)]
df_bern = df_bern[df_bern.StationCode.eq("ABO")]
df_bern = df_bern.reset_index()

# fillnan with zero
df_bern = df_bern.fillna(0)

In [6]:
df_bern.iloc[200:260]

,index,Unnamed: 0,StationCode,Datum,T,H,U,P,FFMC,DMC,DC,FWI,RegionName,Warnstufe,StationName,BUI,ISI,Kanton
200,2674,2675,ABO,2018-07-20,22.0,51.2,3.6,23.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,2,Adelboden,0.0,0.000,0
201,2675,2676,ABO,2018-07-21,13.7,77.7,1.3,8.2,0.0,0.0,0.0,0.0,Kandertal Adelboden,2,Adelboden,0.0,0.000,0
202,2676,2677,ABO,2018-07-22,13.0,82.1,0.9,0.2,0.0,0.0,0.0,0.0,Kandertal Adelboden,2,Adelboden,0.0,0.000,0
203,2677,2678,ABO,2018-07-23,16.3,71.5,3.1,0.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,2,Adelboden,0.0,0.000,0
204,2678,2679,ABO,2018-07-24,21.0,56.9,2.8,9.5,0.0,0.0,0.0,0.0,Kandertal Adelboden,2,Adelboden,0.0,0.000,0
205,2679,2680,ABO,2018-07-25,18.9,66.1,1.6,0.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,3,Adelboden,0.0,0.000,0
206,2680,2681,ABO,2018-07-26,22.8,40.6,3.3,0.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,3,Adelboden,0.0,0.000,0
207,2681,2682,ABO,2018-07-27,22.6,46.2,3.1,0.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,3,Adelboden,0.0,0.000,0
208,2682,2683,ABO,2018-07-28,14.9,91.2,2.2,7.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,3,Adelboden,0.0,0.000,0
209,2683,2684,ABO,2018-07-29,20.6,56.7,3.5,0.0,0.0,0.0,0.0,0.0,Kandertal Adelboden,3,Adelboden,0.0,0.000,0


In [337]:
# first test, calculate FWI components
ffmc0 = 85
for index, row in df_bern.iloc[100:110].iterrows():
    print(index, ": ", row['Datum'])
    print(row['T'],row['P'],row['U'],row['H'],df_bern.iloc[index-1]['FFMC'])
    ffmc = firedanger.ffmc(t=row['T'],p=row['P'],w=row['U'],h=row['H'],ffmc0=df_bern.iloc[index-1]['FFMC'])
    ffmc0 = ffmc
    print(ffmc, "     vs     ", row['FFMC'])

100 :  2020-02-23 00:00:00
0.0 0.0 4.8 0.0 87.39
95.51554226235591      vs      78.9948279
101 :  2020-02-24 00:00:00
15.8 0.0 4.1 17.0 78.9948279
90.32382112069946      vs      90.32649055
102 :  2020-02-24 00:00:00
15.8 0.0 4.1 17.4 90.32649055
92.76573058652725      vs      92.14
103 :  2020-02-25 00:00:00
7.5 0.0 9.3 32.0 92.14
90.82406415247743      vs      90.824
104 :  2020-02-25 00:00:00
7.5 0.0 9.4 32.0 90.824
90.36015827450251      vs      90.18448039
105 :  2020-02-26 00:00:00
-4.0 8.8 2.6 72.0 90.18448039
36.16402670293026      vs      36.294
106 :  2020-02-26 00:00:00
-4.0 8.8 2.6 72.0 36.294
23.140085678616966      vs      36.1786083
107 :  2020-02-27 00:00:00
2.3 5.9 5.5 78.7 36.1786083
31.31656103558368      vs      31.353
108 :  2020-02-27 00:00:00
2.3 5.9 5.5 79.0 31.353
29.648751140563387      vs      31.00094045
109 :  2020-02-28 00:00:00
1.9 8.1 1.6 24.6 31.00094045
36.023931685390416      vs      36.114


# My CODES work fine. 
# lets start building a nice package that automatically reads timeseries (station) and gridded data --> and then calculates the indices


## Step 1: Read files
Tool needs to be able to read gridded data (with dims: time, lat, lon), but also timeseries (with dim: time)

In [8]:
import numpy as np
import xarray as xr
import pandas as pd
import zipfile

In [15]:
# read hourly meteoswiss data for Locarno - Monti (OTL)
datapath = '/home/steidani/windows/data/forest_fire/angelis_2015/'
zf = zipfile.ZipFile(datapath + 'order95434.zip') 
df_mch = pd.read_csv(zf.open('order_95434_data.txt'), sep=";")
df_mch.columns = ["stn", "time", "T", "P", "H", "U"]

In [16]:
# convert column "time" to  type datetime
df_mch['time'] = pd.to_datetime(df_mch['time'], format='%Y%m%d%H', errors='coerce')

In [17]:
# daily values: Here at 12noon
df_mch = df_mch[df_mch.time.dt.hour.eq(12)]
df_mch = df_mch.reset_index()
df_mch.time = df_mch.time.dt.strftime('%Y%m%d')
df_mch

,index,stn,time,T,P,H,U
0,12,OTL,19910501,10.9,4.5,83.6,2.4
1,36,OTL,19910502,11.4,12.8,75.3,1.6
2,60,OTL,19910503,16.0,8.2,45.7,2.0
3,84,OTL,19910504,9.6,0.0,42.0,2.8
4,108,OTL,19910505,10.5,20.2,59.2,4.3
...,...,...,...,...,...,...,...
209,5028,OTL,19911126,9.1,0.0,58.6,1.1
210,5052,OTL,19911127,9.3,0.0,62.3,1.5
211,5076,OTL,19911128,9.3,0.0,64.4,1.3
212,5100,OTL,19911129,10.1,0.0,55.9,1.0


In [19]:
# save
df_mch.to_csv('/home/steidani/windows/data/forest_fire/test.csv', index=False)

In [20]:
ds_mch = pd.read_csv('/home/steidani/windows/data/forest_fire/test.csv')
df_mch

,index,stn,time,T,P,H,U
0,12,OTL,19910501,10.9,4.5,83.6,2.4
1,36,OTL,19910502,11.4,12.8,75.3,1.6
2,60,OTL,19910503,16.0,8.2,45.7,2.0
3,84,OTL,19910504,9.6,0.0,42.0,2.8
4,108,OTL,19910505,10.5,20.2,59.2,4.3
...,...,...,...,...,...,...,...
209,5028,OTL,19911126,9.1,0.0,58.6,1.1
210,5052,OTL,19911127,9.3,0.0,62.3,1.5
211,5076,OTL,19911128,9.3,0.0,64.4,1.3
212,5100,OTL,19911129,10.1,0.0,55.9,1.0


In [43]:
# pandas to xarray
ds = df_mch.set_index(['time']).to_xarray()

In [47]:
ds.time.dt.month

<xarray.DataArray 'month' (time: 214)>
array([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11])
Coordinates:
  * time     (time) datetime64[ns] 1991-05-01T12:00:00 ... 1991-11-30T12:00:00